In [1]:
import cv2
import face_recognition
import numpy as np
import torch
import torch.nn.functional as F
from collections import OrderedDict
from Architecture import anti_spoofing
import time
import dlib

In [ ]:


image1 = face_recognition.load_image_file("1.jpg")
image_encode1 = face_recognition.face_encodings(image1)[0]

image2 = face_recognition.load_image_file("3.jpg")
image_encode2 = face_recognition.face_encodings(image2)[0]

known_faces = [image_encode1, image_encode2]
known_names = ['bassem', 'Soudy']

device = torch.device("cpu")
model = anti_spoofing().to(device)
weights_path = 'model.pth'
state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
new_state_dict = OrderedDict()
for key, value in state_dict.items():
    name_key = key[7:]
    new_state_dict[name_key] = value
model.load_state_dict(new_state_dict)
model.eval()

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()


    detector = dlib.get_frontal_face_detector()
    faces = detector(frame)

    for face in faces:
        x, y, width, height = face.left(), face.top(), face.width(), face.height()
        h, w, c = np.shape(frame)

        scale = min((h - 1) / height, min((w - 1) / width, 2.7))
        new_width = width * scale
        new_height = height * scale
        center_x, center_y = width / 2 + x, height / 2 + y

        left_top_x = center_x - new_width / 2
        left_top_y = center_y - new_height / 2
        right_bottom_x = center_x + new_width / 2
        right_bottom_y = center_y + new_height / 2

        left_top_x = max(0, int(left_top_x))
        left_top_y = max(0, int(left_top_y))
        right_bottom_x = min(w - 1, int(right_bottom_x))
        right_bottom_y = min(h - 1, int(right_bottom_y))

        face_region = frame[left_top_y:right_bottom_y + 1, left_top_x:right_bottom_x + 1]
        face_region = cv2.resize(face_region, (80, 80))
        face_tensor = torch.from_numpy(face_region.transpose((2, 0, 1))).float()
        face_tensor = face_tensor.unsqueeze(0).to(torch.device('cpu'))

        with torch.no_grad():
            output = model.forward(face_tensor)
            result = F.softmax(output, dim=-1).cpu().numpy()

        label = np.argmax(result)
        value = result[0][label]

        if label == 1:
            new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            locations = face_recognition.face_locations(new_frame)
            faces = face_recognition.face_encodings(new_frame, locations)
            
            names = []
            for face in faces:
                compare = face_recognition.compare_faces(known_faces, face)
                similarity = face_recognition.face_distance(known_faces, face)
                if compare[np.argmin(similarity)]:
                    name = known_names[np.argmin(similarity)]
                else:
                    name = "Unknown"
                names.append(name)

            for (y_, h_, w_, x_), name in zip(locations, names):
                cv2.rectangle(frame, (x_, y_), (h_, w_), (255, 0, 0), 2)
                cv2.putText(frame, name, (x_ + 6, y_ - 40), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 255), 1)

        else:
            # print("Fake Face Detected. Score: {:.2f}".format(value))
            cv2.rectangle(frame, (x, y), (x + width, y + height), (255, 0, 0), 2)
            cv2.putText(frame, 'Fake', (x + 6, y - 40), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 255), 1)

        cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import face_recognition
import numpy as np
import torch
import torch.nn.functional as F
from collections import OrderedDict
from Architecture import anti_spoofing
import time

image1 = face_recognition.load_image_file("1.jpg")
image_encode1 = face_recognition.face_encodings(image1)[0]

image2 = face_recognition.load_image_file("3.jpg")
image_encode2 = face_recognition.face_encodings(image2)[0]

known_faces = [image_encode1, image_encode2]
known_names = ['bassem', 'Soudy']


ima = cv2.imread('1.jpg')


device = torch.device("cpu")
model = anti_spoofing().to(device)
weights_path = 'model.pth'
state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
new_state_dict = OrderedDict()
for key, value in state_dict.items():
    name_key = key[7:]
    new_state_dict[name_key] = value
model.load_state_dict(new_state_dict)
model.eval()

detector = dlib.get_frontal_face_detector()
faces = detector(ima)

for face in faces:
    x, y, width, height = face.left(), face.top(), face.width(), face.height()
    h, w, c = np.shape(frame)

    scale = min((h - 1) / height, min((w - 1) / width, 2.7))
    new_width = width * scale
    new_height = height * scale
    center_x, center_y = width / 2 + x, height / 2 + y

    left_top_x = center_x - new_width / 2
    left_top_y = center_y - new_height / 2
    right_bottom_x = center_x + new_width / 2
    right_bottom_y = center_y + new_height / 2

    left_top_x = max(0, int(left_top_x))
    left_top_y = max(0, int(left_top_y))
    right_bottom_x = min(w - 1, int(right_bottom_x))
    right_bottom_y = min(h - 1, int(right_bottom_y))

    face_region = frame[left_top_y:right_bottom_y + 1, left_top_x:right_bottom_x + 1]
    face_region = cv2.resize(face_region, (80, 80))
    face_tensor = torch.from_numpy(face_region.transpose((2, 0, 1))).float()
    face_tensor = face_tensor.unsqueeze(0).to(torch.device('cpu'))

    with torch.no_grad():
        output = model.forward(face_tensor)
        result = F.softmax(output, dim=-1).cpu().numpy()

    label = np.argmax(result)
    value = result[0][label]

    if label == 1:
        new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        locations = face_recognition.face_locations(new_frame)
        faces = face_recognition.face_encodings(new_frame, locations)

        names = []
        for face in faces:
            compare = face_recognition.compare_faces(known_faces, face)
            similarity = face_recognition.face_distance(known_faces, face)
            if compare[np.argmin(similarity)]:
                name = known_names[np.argmin(similarity)]
            else:
                name = "Unknown"
            names.append(name)

        for (y_, h_, w_, x_), name in zip(locations, names):
            cv2.rectangle(frame, (x_, y_), (h_, w_), (255, 0, 0), 2)
            cv2.putText(frame, name, (x_ + 6, y_ - 40), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 255), 1)

    else:
        # print("Fake Face Detected. Score: {:.2f}".format(value))
            cv2.rectangle(frame, (x, y), (x + width, y + height), (255, 0, 0), 2)
            cv2.putText(frame, 'Fake', (x + 6, y - 40), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 255), 1)

   

In [ ]:
plt.imshow(ima)